In [1]:


import pandas as pd
import numpy as np
import os


import library as lib
from library import GlobalVars
from library import ExperimentConfig
import data_library as data_lib

import faiss_experiment as faiss_tool
import helpers.config_executor as executor


from sklearn.model_selection import StratifiedKFold

import experiments_configurations as ex_cfg
#
lib.init_random_seed(lib.RANDOM_SEED)

index_type="ivf_all"
faiss_experiments_output_file = GlobalVars.experiments_path + f"executed_faiss_experiments_index_{index_type}_kfold.csv"
all_faiss_experiments_file_path= GlobalVars.experiments_path + f"all_faiss_experiments_index_{index_type}.csv"
lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, getattr(ex_cfg, index_type)) # ex: ex_cfg.ivf_all




In [2]:
"""
Execute ONE configuration as test calling individual process
"""
def execute_one_configuration_as_process():
    os.environ["LOKY_MAX_CPU_COUNT"] = "8"
    debug_mode=True
    cfg_records=pd.read_csv(all_faiss_experiments_file_path)
    row = cfg_records.iloc[0].copy()
    new_row = executor.launch_execute_configurations_as_process(faiss_tool, row, debug_mode=debug_mode)
    print("results=", new_row)

In [3]:
"""
Execute ONE configuration as test by library function call
"""
def execute_one_configuration():
    debug_mode=True
    cfg_records=pd.read_csv(all_faiss_experiments_file_path)
    row = cfg_records.iloc[0].copy()
    print(row)
    print("index params=", row['index_params'], type(row["index_params"]))
    cfg = ExperimentConfig(row)
    GlobalVars.set_segment_lenght_and_overlap(cfg._SEGMENT_LENGHT, cfg._SEGMENT_OVERLAP)


    faiss_results = faiss_tool.execute_configuration(cfg)
    print("results=", faiss_results)

In [ ]:
'''
Execute all configurations from all_faiss_experiments_file_path
'''
def execute_all_configurations(num_folds=0):
    global all_faiss_experiments_file_path
    global index_type
    do_normalize = True
    debug_mode = False
    repeat_no = 5    

    lib.CACHING_INDEX = False

    local_all_faiss_experiments_file_path= GlobalVars.experiments_path + f"all_faiss_experiments_index_{index_type}_tmp.csv"
    lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, getattr(ex_cfg, index_type)) # ex: ex_cfg.ivf_all
    df = pd.read_csv(all_faiss_experiments_file_path)
    df['num_folds'] = num_folds
    #df = df.head(10)
    df.to_csv(local_all_faiss_experiments_file_path, index=False) 
    print("local_all_faiss_experiments_file_path=", local_all_faiss_experiments_file_path)
    
    for i in range(repeat_no):
        print(f"Execute round {i}")
        
        #you can cache the indexes before if you want
        #faiss_tool.cache_all_faiss_indexes(all_faiss_experiments_file_path)
            
        executor.debug_mod= True
        faiss_experiments_output_file = GlobalVars.experiments_path + f"__executed_faiss_experiments_index_{index_type}_round{i}_kfold{num_folds}.csv"
        executor.process_all_configs_by_threads(exec_tool=faiss_tool, input_file_path=local_all_faiss_experiments_file_path, output_file_path=faiss_experiments_output_file, num_threads = 4, chunk_size = 20)

execute_all_configurations(5)

local_all_faiss_experiments_file_path= /Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../experiments/all_faiss_experiments_index_ivf_all_tmp.csv
Execute round 0


Process: 100%|██████████| 10/10 [03:22<00:00, 20.28s/rows]

Done


In [5]:


# os.environ["LOKY_MAX_CPU_COUNT"] = "8"
# debug_mode=True
# cfg_records=pd.read_csv(all_faiss_experiments_file_path)
# row = cfg_records.iloc[1].copy()
# row["num_folds"] = 5
# cfg = ExperimentConfig(row)
# cfg._USE_CACHE = False
# #cfg._SEGMENT_OVERLAP = 5
# GlobalVars.set_segment_lenght_and_overlap(cfg._SEGMENT_LENGHT, cfg._SEGMENT_OVERLAP)
# # metrics_json = faiss_tool.execute_configuration(cfg)
# # print("==========================================================================")
# # new_row = row.copy()
# results = executor.launch_execute_configurations_as_process(faiss_tool, row, debug_mode=debug_mode)
# # error = ""
# # print(metrics_json)
# # print(metrics_json["accuracy_score"])
# # new_row = lib.extract_experiment_results(new_row=new_row, tool_results=metrics_json, process_key="1111222222", error=error)
# # print("results=", new_row)

# print(results)